<a href="https://colab.research.google.com/github/Muzhi1920/awesome-models/blob/main/05-%E7%89%B9%E5%BE%81%E4%BA%A4%E4%BA%92/02_DCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DCN

- 论文：Deep & Cross Network for Ad Click Predictions
- 参考：https://arxiv.org/pdf/1708.05123.pdf

Cross网络（Cross Network），设计该网络的目的是增加特征之间的交互力度。交叉网络由多个交叉层组成，递归地cross输入：

- 高阶输入，通过交叉得到权重w；
- 权重w乘到inputs上，增加bias；
- 最后再加上inputs，一次交叉完成










In [ ]:
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, Embedding, Conv1D
from tensorflow.keras.regularizers import l2

## 准备工作

In [ ]:
nums = fc.numeric_column('nums', dtype=tf.float32)
seq = fc.categorical_column_with_hash_bucket('seq', hash_bucket_size=10, dtype=tf.int64)
target = fc.categorical_column_with_hash_bucket('target', hash_bucket_size=10, dtype=tf.int64)
seq_col = fc.embedding_column(seq, dimension=8)
target_col = fc.embedding_column(target, dimension=8)
columns = [seq_col, target_col, nums]
features={
    "seq": tf.sparse.SparseTensor(
        indices=[[0, 0], [0, 1], [1, 0], [1, 1], [2, 0]],
        values=[1100, 1101, 1102, 1101, 1103],
        dense_shape=[3, 2]),
    "target": tf.sparse.SparseTensor(
        indices=[[0, 0],[1,0],[2,0]],
        values=[1102,1103,1100],
        dense_shape=[3, 1]),
    "nums": tf.convert_to_tensor([0.1,0.2,0.3]) 

}
tf.sparse.to_dense(features['seq'])

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1100, 1101],
       [1102, 1101],
       [1103,    0]], dtype=int32)>

## 输出tensor

In [ ]:
input_layer = tf.keras.layers.DenseFeatures(columns, name='features_input_layer')
net = input_layer(features)
#tf.concat(sequence_inputs.values(), axis =-1)
net

<tf.Tensor: shape=(3, 17), dtype=float32, numpy=
array([[ 0.1       ,  0.06989709,  0.05039133, -0.40526628, -0.17505139,
        -0.08593822, -0.47402114,  0.24738257, -0.03490505,  0.11353349,
         0.10683289,  0.2729674 , -0.12089346,  0.25551936,  0.17137639,
         0.07327487,  0.5861915 ],
       [ 0.2       ,  0.20494407,  0.20746717, -0.0110372 ,  0.18881464,
         0.14407712, -0.08519235,  0.06070877, -0.10711194,  0.35411718,
        -0.5623867 ,  0.42709053, -0.6976193 ,  0.5282704 , -0.14868078,
         0.34827474, -0.3380999 ],
       [ 0.3       ,  0.07574383,  0.33736795,  0.62267196, -0.37300104,
        -0.31420803, -0.27134705,  0.06302836,  0.25845692,  0.10021805,
         0.0760479 , -0.6557216 ,  0.5330804 , -0.41521493,  0.082963  ,
         0.21762002,  0.0465186 ]], dtype=float32)>

## deep cross network

In [ ]:
#DCN config
layer_num = 2
reg_w = 1e-6
reg_b = 1e-6

input_dims = tf.shape(net)[-1]

cross_weights = [Dense(name='w_' + str(i),
                       units=1,
                       trainable=True,
                       use_bias=True)
                for i in range(layer_num)]

def cross_layer(x_0):
    x_l = x_0
    for i in range(layer_num):
        x_l_weight = cross_weights[i](x_l)
        x_l = x_0 * x_l_weight + x_l
    return x_l



In [ ]:
cross_layer(net)

<tf.Tensor: shape=(3, 17), dtype=float32, numpy=
array([[ 0.09316738,  0.06512129,  0.04694828, -0.377576  , -0.1630908 ,
        -0.08006638, -0.4416331 ,  0.23047987, -0.03252012,  0.10577618,
         0.0995334 ,  0.25431657, -0.11263326,  0.2380607 ,  0.1596669 ,
         0.06826827,  0.54613924],
       [ 0.31724018,  0.32508245,  0.32908458, -0.01750722,  0.2994979 ,
         0.22853523, -0.13513216,  0.09629629, -0.16990103,  0.56170094,
        -0.8920582 ,  0.6774513 , -1.1065643 ,  0.83794296, -0.23583755,
         0.55243367, -0.53629434],
       [ 0.27403557,  0.06918834,  0.3081694 ,  0.5687809 , -0.3407185 ,
        -0.28701395, -0.24786246,  0.05757337,  0.23608798,  0.09154437,
         0.0694661 , -0.5989702 ,  0.4869433 , -0.37927884,  0.07578272,
         0.19878542,  0.0424925 ]], dtype=float32)>